In [5]:
from __future__ import print_function

import os
import sys
import glob
import h5py
import numpy as np
import math


import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset , DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from tqdm import tqdm


from log import Logger
from data import gDataset, trainDataset, testDataset
from util import r2, mse, rmse, mae, pp_mse, pp_rmse, pp_mae
#from model import autoencoder_0, autoencoder_1, autoencoder_2 

In [6]:
if not os.path.exists('./gal_img'):
    os.mkdir('./gal_img')

    
    
def to_img(x):
    x = x.view(x.size(0), 1, 96, 96)
    return x

num_epochs =1000
batch_size = 64
learning_rate = 1e-4


def plot_sample_img(img, name):
    img = img.view(1, 96, 96)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)






In [30]:
class autoencoder_2(nn.Module):
    def __init__(self):
        super(autoencoder_2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2), 
            
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1),  
            
            nn.Conv2d(8, 2, 3, stride=1, padding=1),  # b, 2,
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1),  # b, 2,
            
            nn.Conv2d(2, 1, 3, stride=2, padding=1),  # b, 1, 3, 3 
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(1, 2, 3, stride=1, padding =1 ),  # b, 2, 
            nn.ReLU(True),
            
            nn.ConvTranspose2d(2, 8, 3, stride=1),  # b, 8, 
            nn.ReLU(True),
            
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16,  
            nn.ReLU(True),
            
            nn.ConvTranspose2d(16, 8, 4, stride=2, padding = 1 ),  # b, 8,
            nn.ReLU(True),
                       
            nn.ConvTranspose2d(8, 4, 4, stride=2, padding = 1 ),  # b, 8,
            nn.ReLU(True),
            
            nn.ConvTranspose2d(4, 1, 10, stride=2),  # b, 1, 96, 96
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
dataset= gDataset()
dataloader= DataLoader(dataset=dataset, batch_size=64,shuffle=True)



model = autoencoder_2().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=learning_rate, weight_decay=1e-5)


for epoch in range(num_epochs):
    for data in dataloader:
        img = data
        
        
        
        img.type(torch.float32)

        img = img.view(img.size(0), 1,96,96)
        img = Variable(img).cuda().type('torch.FloatTensor')
        
        #print(img.shape)
        

        # forward

        output = model(img.cuda())
        
      #  print("output ",output.shape)




        loss = criterion(output.cuda(), img.cuda())
        
        MSE_loss = nn.MSELoss()(output.cuda(), img.cuda())
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # log
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data, MSE_loss.data))
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        torch.save(x, './s2_x_{}.pt'.format(epoch))
        torch.save(x_hat, './s2_x_hat_{}.pt'.format(epoch))

torch.save(model.state_dict(), './s2_sim_autoencoder.pth')

epoch [1/1000], loss:0.6507, MSE_loss:0.2263
epoch [2/1000], loss:0.2848, MSE_loss:0.0680
epoch [3/1000], loss:0.1239, MSE_loss:0.0270
epoch [4/1000], loss:0.1116, MSE_loss:0.0208
epoch [5/1000], loss:0.0919, MSE_loss:0.0175
epoch [6/1000], loss:0.0924, MSE_loss:0.0151
epoch [7/1000], loss:0.0942, MSE_loss:0.0131
epoch [8/1000], loss:0.0884, MSE_loss:0.0107
epoch [9/1000], loss:0.0695, MSE_loss:0.0084
epoch [10/1000], loss:0.0626, MSE_loss:0.0067
epoch [11/1000], loss:0.0559, MSE_loss:0.0049
epoch [12/1000], loss:0.0509, MSE_loss:0.0034
epoch [13/1000], loss:0.0335, MSE_loss:0.0022
epoch [14/1000], loss:0.0341, MSE_loss:0.0018
epoch [15/1000], loss:0.0302, MSE_loss:0.0018
epoch [16/1000], loss:0.0298, MSE_loss:0.0013
epoch [17/1000], loss:0.0344, MSE_loss:0.0012
epoch [18/1000], loss:0.0224, MSE_loss:0.0010
epoch [19/1000], loss:0.0247, MSE_loss:0.0011
epoch [20/1000], loss:0.0216, MSE_loss:0.0008
epoch [21/1000], loss:0.0203, MSE_loss:0.0008
epoch [22/1000], loss:0.0212, MSE_loss:0.00

epoch [178/1000], loss:0.0150, MSE_loss:0.0002
epoch [179/1000], loss:0.0223, MSE_loss:0.0003
epoch [180/1000], loss:0.0143, MSE_loss:0.0002
epoch [181/1000], loss:0.0194, MSE_loss:0.0006
epoch [182/1000], loss:0.0187, MSE_loss:0.0002
epoch [183/1000], loss:0.0125, MSE_loss:0.0002
epoch [184/1000], loss:0.0195, MSE_loss:0.0003
epoch [185/1000], loss:0.0150, MSE_loss:0.0003
epoch [186/1000], loss:0.0154, MSE_loss:0.0002
epoch [187/1000], loss:0.0198, MSE_loss:0.0004
epoch [188/1000], loss:0.0185, MSE_loss:0.0004
epoch [189/1000], loss:0.0136, MSE_loss:0.0002
epoch [190/1000], loss:0.0139, MSE_loss:0.0002
epoch [191/1000], loss:0.0143, MSE_loss:0.0002
epoch [192/1000], loss:0.0133, MSE_loss:0.0002
epoch [193/1000], loss:0.0186, MSE_loss:0.0003
epoch [194/1000], loss:0.0153, MSE_loss:0.0003
epoch [195/1000], loss:0.0169, MSE_loss:0.0004
epoch [196/1000], loss:0.0203, MSE_loss:0.0005
epoch [197/1000], loss:0.0208, MSE_loss:0.0003
epoch [198/1000], loss:0.0225, MSE_loss:0.0002
epoch [199/10

epoch [353/1000], loss:0.0185, MSE_loss:0.0003
epoch [354/1000], loss:0.0153, MSE_loss:0.0001
epoch [355/1000], loss:0.0169, MSE_loss:0.0002
epoch [356/1000], loss:0.0156, MSE_loss:0.0004
epoch [357/1000], loss:0.0214, MSE_loss:0.0003
epoch [358/1000], loss:0.0155, MSE_loss:0.0002
epoch [359/1000], loss:0.0199, MSE_loss:0.0003
epoch [360/1000], loss:0.0167, MSE_loss:0.0002
epoch [361/1000], loss:0.0128, MSE_loss:0.0002
epoch [362/1000], loss:0.0194, MSE_loss:0.0003
epoch [363/1000], loss:0.0187, MSE_loss:0.0003
epoch [364/1000], loss:0.0135, MSE_loss:0.0002
epoch [365/1000], loss:0.0206, MSE_loss:0.0004
epoch [366/1000], loss:0.0138, MSE_loss:0.0002
epoch [367/1000], loss:0.0140, MSE_loss:0.0004
epoch [368/1000], loss:0.0143, MSE_loss:0.0002
epoch [369/1000], loss:0.0126, MSE_loss:0.0002
epoch [370/1000], loss:0.0178, MSE_loss:0.0002
epoch [371/1000], loss:0.0138, MSE_loss:0.0002
epoch [372/1000], loss:0.0148, MSE_loss:0.0002
epoch [373/1000], loss:0.0139, MSE_loss:0.0002
epoch [374/10

epoch [528/1000], loss:0.0147, MSE_loss:0.0002
epoch [529/1000], loss:0.0162, MSE_loss:0.0002
epoch [530/1000], loss:0.0130, MSE_loss:0.0002
epoch [531/1000], loss:0.0129, MSE_loss:0.0002
epoch [532/1000], loss:0.0172, MSE_loss:0.0005
epoch [533/1000], loss:0.0184, MSE_loss:0.0003
epoch [534/1000], loss:0.0181, MSE_loss:0.0003
epoch [535/1000], loss:0.0155, MSE_loss:0.0002
epoch [536/1000], loss:0.0162, MSE_loss:0.0002
epoch [537/1000], loss:0.0154, MSE_loss:0.0002
epoch [538/1000], loss:0.0158, MSE_loss:0.0003
epoch [539/1000], loss:0.0153, MSE_loss:0.0003
epoch [540/1000], loss:0.0144, MSE_loss:0.0002
epoch [541/1000], loss:0.0247, MSE_loss:0.0003
epoch [542/1000], loss:0.0155, MSE_loss:0.0002
epoch [543/1000], loss:0.0131, MSE_loss:0.0002
epoch [544/1000], loss:0.0130, MSE_loss:0.0002
epoch [545/1000], loss:0.0179, MSE_loss:0.0002
epoch [546/1000], loss:0.0126, MSE_loss:0.0002
epoch [547/1000], loss:0.0195, MSE_loss:0.0003
epoch [548/1000], loss:0.0140, MSE_loss:0.0004
epoch [549/10

epoch [703/1000], loss:0.0117, MSE_loss:0.0002
epoch [704/1000], loss:0.0122, MSE_loss:0.0002
epoch [705/1000], loss:0.0143, MSE_loss:0.0002
epoch [706/1000], loss:0.0225, MSE_loss:0.0028
epoch [707/1000], loss:0.0176, MSE_loss:0.0004
epoch [708/1000], loss:0.0102, MSE_loss:0.0002
epoch [709/1000], loss:0.0186, MSE_loss:0.0002
epoch [710/1000], loss:0.0166, MSE_loss:0.0002
epoch [711/1000], loss:0.0134, MSE_loss:0.0002
epoch [712/1000], loss:0.0176, MSE_loss:0.0003
epoch [713/1000], loss:0.0162, MSE_loss:0.0002
epoch [714/1000], loss:0.0127, MSE_loss:0.0004
epoch [715/1000], loss:0.0139, MSE_loss:0.0002
epoch [716/1000], loss:0.0144, MSE_loss:0.0002
epoch [717/1000], loss:0.0211, MSE_loss:0.0002
epoch [718/1000], loss:0.0216, MSE_loss:0.0011
epoch [719/1000], loss:0.0236, MSE_loss:0.0014
epoch [720/1000], loss:0.0278, MSE_loss:0.0012
epoch [721/1000], loss:0.0192, MSE_loss:0.0002
epoch [722/1000], loss:0.0156, MSE_loss:0.0002
epoch [723/1000], loss:0.0135, MSE_loss:0.0002
epoch [724/10

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out
    
    
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)

        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out
class Encoder(nn.Module):

    def __init__(self, block, layers, num_classes=23):
        self.inplanes = 64
        super (Encoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)#, return_indices = True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, 1000)
	#self.fc = nn.Linear(num_classes,16) 
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
	
        x = self.bn1(x)
        x = self.relu(x)
	
        x = self.maxpool(x)
	
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
    
encoder = Encoder(Bottleneck, [3, 4, 6, 3])

encoder.fc = nn.Linear(2048, 48)

encoder=encoder.cuda()
y=torch.rand(1,3,224,224)
x=torch.rand(1,128)
x=Variable(x.cuda())

zsize=10  
    
class Decoder(nn.Module):
	def __init__(self):
		super(Decoder,self).__init__()
		self.dfc3 = nn.Linear(zsize, 4096)
		self.bn3 = nn.BatchNorm2d(4096)
		self.dfc2 = nn.Linear(4096, 4096)
		self.bn2 = nn.BatchNorm2d(4096)
		self.dfc1 = nn.Linear(4096,256 * 6 * 6)
		self.bn1 = nn.BatchNorm2d(256*6*6)
		self.upsample1=nn.Upsample(scale_factor=2)
		self.dconv5 = nn.ConvTranspose2d(256, 256, 3, padding = 0)
		self.dconv4 = nn.ConvTranspose2d(256, 384, 3, padding = 1)
		self.dconv3 = nn.ConvTranspose2d(384, 192, 3, padding = 1)
		self.dconv2 = nn.ConvTranspose2d(192, 64, 5, padding = 2)
		self.dconv1 = nn.ConvTranspose2d(64, 3, 12, stride = 4, padding = 4)

	def forward(self,x):#,i1,i2,i3):
		
		x = self.dfc3(x)
		#x = F.relu(x)
		x = F.relu(self.bn3(x))
		
		x = self.dfc2(x)
		x = F.relu(self.bn2(x))
		#x = F.relu(x)
		x = self.dfc1(x)
		x = F.relu(self.bn1(x))
		#x = F.relu(x)
		#print(x.size())
		x = x.view(batch_size,256,6,6)
		#print (x.size())
		x=self.upsample1(x)
		#print x.size()
		x = self.dconv5(x)
		#print x.size()
		x = F.relu(x)
		#print x.size()
		x = F.relu(self.dconv4(x))
		#print x.size()
		x = F.relu(self.dconv3(x))
		#print x.size()		
		x=self.upsample1(x)
		#print x.size()		
		x = self.dconv2(x)
		#print x.size()		
		x = F.relu(x)
		x=self.upsample1(x)
		#print x.size()
		x = self.dconv1(x)
		#print x.size()
		x = F.sigmoid(x)
		#print x
		return x
decoder = Decoder()




class autoencoder(nn.Module):
	def __init__(self):
		super(autoencoder,self).__init__()
		self.encoder = encoder(x)
		self.decoder = decoder()

	def forward(self,x):
		#x=Encoder(x)
		x = self.encoder(x)

		x = self.decoder(x)
		return x
    
    
    
    
    
 